### Integracion trayectoria de meteoro del paper:  A Meteor of Apparent Interstellar Origin in the CNEOS Fireball Catalog (Amir Siraj and Abraham Loeb)

 https://doi.org/10.3847/1538-4357/ac8eac

In [1]:
import pandas as pd
from astropy.time import Time
import spiceypy as spy
import numpy as np
import rebound as rb
import matplotlib.pyplot as plt
from Utils import *
from astroquery.jplhorizons import Horizons

Se cargan los datos del catalogo de fireballs de CNEOS

In [2]:
fireballs = pd.read_csv('datos/cneos_fireball_data_original.csv', comment='#')
fireballs

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-08-05 19:54:50,19.9N,131.7W,44.0,21.8,3.3,20.4,-6.8,1.719000e+12,3.800
1,2023-07-26 03:41:54,18.9N,103.4W,36.1,15.6,-9.5,12.2,1.9,1.510000e+11,0.440
2,2023-07-07 16:15:31,14.5N,126.6W,38.3,25.7,2.6,4.1,-25.2,2.600000e+10,0.092
3,2023-07-06 05:52:31,59.3S,145.4W,71.0,NaN,NaN,NaN,NaN,3.000000e+10,0.100
4,2023-06-21 19:39:13,12.4N,62.8E,40.4,16.5,7.3,-2.0,-16.4,5.430000e+11,1.400
...,...,...,...,...,...,...,...,...,...,...
955,1993-11-29 17:48:41,26.5N,78.3E,NaN,NaN,NaN,NaN,NaN,2.600000e+10,0.092
956,1993-10-31 03:39:27,51.3N,100.9W,NaN,NaN,NaN,NaN,NaN,4.000000e+10,0.130
957,1991-10-04 09:22:47,78.7N,6.3E,NaN,NaN,NaN,NaN,NaN,5.500000e+11,1.400
958,1990-10-01 03:51:47,7.5N,142.8E,NaN,NaN,NaN,NaN,NaN,2.500000e+12,5.200


Localizamos el impacto del meteoro por la fecha

In [3]:
meteor = fireballs.loc[fireballs['Peak Brightness Date/Time (UT)'].str.contains('2014-01-08 17:05:34')]
meteor

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
355,2014-01-08 17:05:34,1.3S,147.6E,18.7,44.8,-3.4,-43.5,-10.3,3.100000e+10,0.11


In [4]:
def change_coord(x):
    #funcion para trasformar el formato de coordenadas terrestres que da CNEOS
    if x[-1] == 'N' or x[-1] == 'E':
        new = float(x[:-1])
    elif x[-1] == 'S' or x[-1] == 'W':
        new = -float(x[:-1])
    return new    

#Parametros que obtenemos de los datos:
date = meteor['Peak Brightness Date/Time (UT)'][355]
lon = change_coord(meteor['Longitude (deg.)'][355])
lat = change_coord(meteor['Latitude (deg.)'][355])
alt = meteor['Altitude (km)'][355]
vx = meteor['vx'][355]
vy = meteor['vy'][355]
vz = meteor['vz'][355]

date, lon, lat, alt, vx, vy, vz

('2014-01-08 17:05:34', 147.6, -1.3, 18.7, -3.4, -43.5, -10.3)

Cargamos los Kernes para hacer la trasfromacion de coordenadas geograficas a posicion en el frame ECLIPTICJ2000

In [5]:
path = 'datos/kernels/'
spy.furnsh([path + 'naif0012.tls', path + 'pck00010.tpc', path + 'earth_fixed.tf', path + 'earth_720101_230601.bpc', path + 'earth_latest_high_prec.bpc'])

Usamos la funcion Geo2Ecliptic de Utils para hacer la trasformacion, podemos trasformas desde varios marcos de referencia terrestres al ECLIPTICJ200, usemos ITRF93 y IAU_EARTH

In [6]:
r_irtf = Geo2Rec(lon, lat, alt) 
r_eclip_itrf93 = Geo2Eclip(lon, lat, alt, date, frame='ITRF93')
r_eclip_iau = Geo2Eclip(lon, lat, alt, date, frame='IAU_EARTH')

print("Position ITRF93 frame: ", r_irtf)
print("Position ITRF93 - ECLIPTICJ200 frame: ", r_eclip_itrf93)
print("Position IAU - ECLIPTICJ200 frame: ", r_eclip_iau)

Position ITRF93 frame:  [-5399.64687765  3426.72010848  -144.1587164 ]
Position ITRF93 - ECLIPTICJ200 frame:  [-5645.85195081  2702.10172644 -1319.99274628]
Position IAU - ECLIPTICJ200 frame:  [-5642.44156711  2707.87470299 -1322.7420744 ]


Debemos trasformar el vector velocidad 

In [7]:
v = np.array([vx, vy, vz])   #meteor's pre-impact velocity in a geocentric Earth-fixed reference frame

T_earth = 86400  #periodo tierra en segundos

#vector velocidad de rotación
omega = np.array([0,0,(2*np.pi)/T_earth]) 

print('Velocidad del asteroide antes del impacto respecto a la tierra ')
print(v, mag(v))  

v_E = (v - spy.vcrss(omega, r_irtf))
#v_E, -v, mag(v_E), np.arccos((v@r_irtf)/(np.linalg.norm(v)*np.linalg.norm(r_irtf)))*180/np.pi

et = spy.utc2et(date)
mx_iau = spy.pxform('IAU_EARTH', 'ECLIPJ2000', et)
mx_itrf = spy.pxform('ITRF93', 'ECLIPJ2000', et)
v_eclip_itrf = spy.mxv(mx_itrf, v_E)
v_eclip_iau = spy.mxv(mx_iau, v_E)

print(f"Velocidad asteroide inercial ecliptico (ITRF93): {v_eclip_itrf}")
print(f"Magnitud velocidad (ITRF93): {mag(v_eclip_itrf)}")

print(f"Velocidad asteroide inercial ecliptico (IAU): {v_eclip_iau}")
print(f"Magnitud velocidad (IAU): {mag(v_eclip_iau)}")


Velocidad del asteroide antes del impacto respecto a la tierra 
[ -3.4 -43.5 -10.3] 44.83190827970632
Velocidad asteroide inercial ecliptico (ITRF93): [  0.13745035 -43.75295533   7.74074714]
Magnitud velocidad (ITRF93): 44.432636191546514
Velocidad asteroide inercial ecliptico (IAU): [  0.08867015 -43.75276672   7.7425255 ]
Magnitud velocidad (IAU): 44.4326361915465


In [8]:
Horizons(id = '399', location='@0', epochs = dict(start = '2014-03-21', stop = '2014-03-22', step = '1d')).vectors()

targetname,datetime_jd,datetime_str,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str11,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64
Earth (399),2456737.5,A.D. 2014-Mar-21 00:00:00.0000,-0.9945468211497613,-0.003722207387582296,-0.0001020957168866397,-0.0002507165931815516,-0.01726151078274061,-1.703512565068483e-07,0.005744063655928266,0.9945537917631541,0.0003153176165069349
Earth (399),2456738.5,A.D. 2014-Mar-22 00:00:00.0000,-0.9946489028189718,-0.0209832991638009,-0.0001022309209574547,4.656936758353987e-05,-0.01725985104532338,-9.456438268899956e-08,0.005745891177136442,0.9948702172514479,0.0003174770455735695


Para realizar la integracion, elijamos trabajar con la trasformacion IAU-ECLIPJ2000

In [9]:
rb.horizons.SSL_CONTEXT = 'unverified' #saca error si no hago esto uwu

sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
sim.integrator = "ias15"

sun = sim.add("Sun", hash='sun', date=date)
earth = sim.add("Earth", hash='earth', date=date)

r_earth = np.array(sim.particles['earth'].xyz)
v_earth = np.array(sim.particles['earth'].vxyz)

r_asteroid = r_eclip_iau  + r_earth 
v_asteroid = v_eclip_iau + v_earth #así si es 

#v_asteroid = -v_eclip + v_earth #esto es errado 
v_esc = (2*sim.G*1.98e30/((r_asteroid@r_asteroid)**0.5))**0.5

asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                   vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2],)
r_asteroid, v_asteroid, r_earth, v_earth, v_esc, mag(v_asteroid)

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')


(array([-4.56682168e+07,  1.39459114e+08, -1.96792418e+04]),
 array([-28.69279537, -53.1376722 ,   7.74260786]),
 array([-4.56625744e+07,  1.39456406e+08, -1.83564997e+04]),
 array([-2.87814655e+01, -9.38490548e+00,  8.23593561e-05]),
 42.438492349511534,
 60.883796606945346)

In [10]:
v_esc_E = (2*sim.G*5.98e24/((r_irtf@r_irtf)**0.5))**0.5
v_mag = mag(v_eclip_iau)
v_inf_E = (v_mag**2 - v_esc_E**2)**0.5
v_esc_E, v_inf_E, v_mag

(11.170665203719915, 43.0055275265486, 44.4326361915465)

In [11]:
sim.status()

---------------------------------
REBOUND version:     	4.4.7
REBOUND built on:    	Mar  9 2025 20:56:14
Number of particles: 	3
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.particle.Particle object at 0x1fc905433d0, m=1.9884754159566474e+30 x=154505.2265538892 y=-338056.7291831016 z=-14068.46131306503 vx=0.01044665676699867 vy=0.004186721288873228 vz=-0.0002423673964135767>
<rebound.particle.Particle object at 0x1fc90541fd0, m=6.045825576341311e+24 x=-45662574.40348314 y=139456406.3682046 z=-18356.49972175807 vx=-28.78146551981982 vy=-9.384905480708625 vz=8.235935607459055e-05>
<rebound.particle.Particle object at 0x1fc905433d0, m=0.0 x=-45668216.84505025 y=139459114.24290758 z=-19679.241796156795 vx=-28.692795370927715 vy=-53.1376721976239 vz=7.74260786142672>
---------------------------------
The following fields have non-default values:
G:
< 1.000000e+00
---
> 6.674080e-20
N:
< 0
-

In [12]:
sim.move_to_com()
o = sim.orbits(primary=sun)
o[1].e, o[1].a/1.496e8  #Verdaderos elementos orbitales (correctos)

(2.3735204241631296, -0.46600352883966073)

Determine the set of osculating conic orbital elements that
corresponds to the state (position, velocity) of a body at
some epoch.

In [13]:
state = [r_eclip_iau[0], r_eclip_iau[1], r_eclip_iau[2], v_eclip_iau[0], v_eclip_iau[1], v_eclip_iau[2]]
mu = 1.98847e30*sim.G
spy.oscelt(state, et, mu)

array([2.41464109e-01, 9.99924509e-01, 2.27512896e-01, 3.84440267e+00,
       1.98165079e+00, 3.15413678e+00, 4.42472801e+08, 1.32712079e+11])

In [14]:
state = [r_asteroid[0], r_asteroid[1], r_asteroid[2], v_asteroid[0], v_asteroid[1], v_asteroid[2]]
mu = 1.98847e30*sim.G
spy.oscelt(state, et, mu)

array([ 9.54224324e+07,  2.36477371e+00,  1.74822513e-01,  1.88801691e+00,
        1.04370242e+00, -1.23379674e+00,  4.42472801e+08,  1.32712079e+11])